In [ ]:
# Import required modules
import subprocess

## Step 1: Running `a1_SmolVLM-Instruct.py`
This step processes the images using `a1_SmolVLM-Instruct.py`. The script generates a description of each image and saves the output in a file called `_output.csv`.

SmolVLM python script uses the Staedel images to describes the image description with generative A.I.. 

In [ ]:
python a1_SmolVLM-Instruct.py image_path_or_folder 
“_output.csv”

## Step 2: Running `a2_metadata_keyword_extract.py`
Then the human Staedel’s XML metadata files keywords are extracted by categories motiv, picture element, association, atmosphere and emotion. The keywords are extracted to one summary file. 

In [ ]:
python a2_metadata_keyword_extract.py path_to_XML_file_or_folder
”metadata_keyword_extract_complete.csv"

## Step 3: Running `3_t5-large-summary-keywordgen.py`
This code processes CSV files containing text descriptions by generating a concise, 125-character summary using the T5-large model and extracting key descriptive words using spaCy. It classifies adjectives into categories—Atmosphere, Emotion, Motive, and Association—using a BART-large zero-shot classifier, and also extracts noun chunks as picture elements. The final analysis, which includes the summary and categorized keywords, is then saved to an output text file "_output_analysis.txt".

In [ ]:
python a3_t5-large-summary-keywordgen.py ”output.csv” 
“_output_analysis.txt”

## Step 4: Running `4_cross_encoder_metadata.py`
This code extracts keyword queries from text files by scanning for specific section headers like "Keywords for Atmosphere:", "Keywords for Emotion:", "Picture Elements:", "Keywords for Association:", and "Keywords for Motive:". It then maps each section to a corresponding category, retrieves matching metadata from a CSV file, and uses a CrossEncoder model to score the semantic similarity between each keyword query and the associated metadata entries. Finally, it writes a summary of the scoring results to a tab-delimited CSV file.

In [ ]:
python 4_cross_encoder_metadata.py path/to/your/text_folder_or_file path/to/your/metadata_keyword_extract_complete.csv
“cross_encoded_summary.csv”

## Step 5: Running `5_cumulated_results_threshold.py`
The threshold of the keywords from cross_encoded_summary.csv are being cut off at the percentage 0.50 and the higher values are still remaining. In the end we have the significant positive results and high outliers.

In [ ]:
python 5_cumulated_results_threshold.py "cross_encoded_summary.csv"
cross_encoded_summary_threshold.csv

## Step 6: Running `6_boxplot_total.py`
This shows a boxplot image for each category atmosphere, emotion and picture el-ements and their distribution on a scale. Since we use cross_encoded_summary_threshold.csv with a threshold of 0.50 percentage, there are no entries below 0.49 percentage.

In [ ]:
python 6_boxplot_total.py cross_encoded_summary_threshold.csv
cumulated_results_boxplot.png

## Step 7: Running `7_metadata_keyword_extract_single.py`
This 7_metadata_keyword_extract_single.py code is an alternative code from 2_metadata_keyword_extract.py that has as an input XML files in a folder and gives as an output for each csv files each image metadata. 2_metadata_keyword_extract.py gives out one summarized file with all keywords. We will use the single csv files for the next step.

In [ ]:
python 7_metadata_keyword_extract_single.py path/to/your/metadata/xml/folder thumb-xl.xml
"thumb-xl.csv"

## Step 8: Running `8_matching_last_keywords.py`
This code processes a collection of CSV files and a threshold file to link keywords with their occurrences in the CSVs. It first scans every CSV file in a specified folder, reading each file's "English Term" and "Percentage" columns. For each CSV row, the code converts the English term to lower-case and builds a dictionary where each keyword maps to a list of tuples containing the source filename and the corresponding percentage.

Next, the code reads a threshold file line by line. It looks for lines that start with a floating-point number followed by a keyword (for example, "0.58 heaviness"). When such a line is found, it checks if the keyword exists in the constructed dictionary. If a match is found, the code appends a formatted string listing all CSV occurrences (filenames and percentages) where that keyword appears to the line; if no match is found, the line remains unchanged. Finally, the modified content is written to a new output file, updating the threshold file with information about the CSV matches.

In [ ]:
python 8_matching_last_keywords.py cross_encoded_summary_threshold.csv sub-sample100_seed=42/metadata
updated_cross_encoded_summary.txt

## Step 9: Running `9_matching_processing_results_file_name.py`
This script reorganizes an earlier analysis summary by grouping keyword match results according to the originating CSV file and specific keyword categories. When it begins reading the file, it checks for lines that indicate which output file is currently being processed. From a line like "Processing file: ...", it extracts and converts the output analysis filename into a corresponding CSV filename. As it continues, encountering lines that start with "== " signals a new category section, such as "Keywords for Atmosphere", "Keywords for Emotion", "Elements of the Picture", "Keywords for Association", or "Keywords for Motive".

The script then processes data lines, which start with a percentage, followed by a keyword, and conclude with a segment enclosed in "[Found: ...]". Using a regular expression, it extracts the percentage value, the keyword, and the list of CSV files (with their associated percentages) where that keyword was detected. It then confirms that the current CSV file (identified earlier) appears in the list from the "[Found: ...]" section; if it does, the script records the keyword and its percentage, ensuring that if the same keyword appears more than once, only the instance with the highest percentage is kept.

After processing all lines, the script aggregates the results in a structured dictionary keyed first by CSV and then by category. For each CSV file, it compiles a report that starts with a header and display of the total number of unique matches. It then processes each of the five categories in a predefined order, listing the keywords sorted in descending order by their percentage scores. Finally, this filtered and grouped report is written to an output file, providing a concise overview of the strongest keyword matches within each category for every processed CSV file.

In [ ]:
python 9_matching_processing_results_file_name.py updated_cross_encoded_summary.txt
final_grouped_results.txt